## Homework 4: StyleGAN 💇🏻‍♂️

С момента своего появления в 2014 году генератор в GAN-ах работал как черный ящик. В традиционных архитектурах скрытый вектор $\mathbf{z}$ подавался в первый слой сети, и, проходя через цепочку сверток, превращался в изображение. Хотя качество изображений росло, исследователи столкнулись с проблемой так называемой **запутанности** (**entanglement**) скрытого пространства. То есть попытка изменить одну характеристику лица неизбежно влекла за собой нежелательные изменения других атрибутов.

Прорывом, предшествующим появлению `StyleGAN`, стала идея **прогрессивного роста** (**Progressive Growing**), предложенная командой `NVIDIA` в модели [ProGAN](https://arxiv.org/pdf/1710.10196). Идея заключалась в том, чтобы начинать обучение с изображений низкого разрешения, постепенно добавляя новые слои и повышая детализацию. Это позволило стабилизировать обучение и достичь высокого разрешения $1024\times 1024$. Однако, несмотря на высокое качество, в `ProGAN` оставалась проблема запутанности скрытого пространства.

Авторы [StyleGAN](https://arxiv.org/pdf/1812.04948) оставили проверенную идею `ProGAN`, но радикально изменили устройство генератора. Вдохновившись методами **переноса стиля** (**Style Transfer**), авторы предложили полностью отказаться от подачи скрытого вектора $\mathbf{z}$ в первый слой нейросети. Вместо этого они предложили начинать с обучаемого тензора,а управление процессом генерации осуществлять через отдельную **сеть отображения** (**Mapping Network**), которая преобразует исходный латентный вектор из искривленного пространства $\mathcal{Z}$ в промежуточное, распутанное пространство $\mathcal{W}$.

### Задание

Вам предстоит реализовать модель `StyleGAN` и обучить модель на датасете `CelebA HQ` для решения задачи генерации лиц.

За выполнение домашнего задания можно получить до **11 баллов**. Для части заданий мы написали для вас скелет. Заполните в них пропуски, выделенные с помощью `...`.

In [ ]:
import copy
import os
import random
from collections import OrderedDict
import zipfile

import gdown
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image 
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms

### Задание 1: Dataset (0.5 балла)

Для обучения нашей модели мы будем использовать датасет `CelebA HQ`, который содержит $30$ тыс. изображений лиц знаменитостей в разрешении $128\times 128$ пикселей.

**Ваша задача**:

- Скачать и распаковать архив с датасетом

- Создать преобразования для всех изображений: 
    - преобразовать в тензор (`ToTensor`).
    - сделать нормализацию (`Normalize`)

- Реализовать и создать класс CelebADataset.

- Разделить созданный датасет на обучающую и валидационную выборки в соотношении $95\% / 5\%$

- Создать DataLoader'ы для каждой выборки.

In [ ]:
def download_and_unzip(file_id, archive_path, target_dir):
    if os.path.isdir(target_dir) and os.listdir(target_dir):
        print(f"Directory '{target_dir}' is not empty. Skipping download.")

    os.makedirs(os.path.dirname(archive_path), exist_ok=True)
    os.makedirs(target_dir, exist_ok=True)

    print(f"Downloading archive ID {file_id} to {archive_path}...")
    gdown.download(id=file_id, output=archive_path, quiet=False, fuzzy=True)
    print("Download complete.")
    
    print(f"Unzipping archive '{archive_path}' to '{target_dir}'...")
    with zipfile.ZipFile(archive_path, 'r') as zip_ref:
        zip_ref.extractall(target_dir) 
    print("Unzipping complete.")
    
    os.remove(archive_path)
    print(f"Archive '{archive_path}' deleted.")
    
DATASET_ID = "1bnYNmmMpb0eXr028qDzB9jHNUwJTCghl" 
TEMP_DIR = "data/temp" 
ARCHIVE_PATH = os.path.join(TEMP_DIR, "celeba_128.zip")
FINAL_IMAGE_DIR = "data/celeba_hq/" 

download_and_unzip(DATASET_ID, ARCHIVE_PATH, FINAL_IMAGE_DIR)

In [ ]:
class CelebADataset(Dataset):
    def __init__(self, root_dir: str, transform=None):
        self.transform = transform
        self.image_paths = ...
        
    def __len__(self):
        ...
    def __getitem__(self, idx):
        ...

In [ ]:
generator = torch.Generator().manual_seed(42) # for train test split

#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ 
# Your code here

train_dataset, val_dataset = ...

Взглянем на наши данные.

In [ ]:
indices = random.sample(range(len(train_dataset)), 20)
fig, axes = plt.subplots(4, 5, figsize=(15, 12))
axes = axes.flatten()

for i, idx in enumerate(indices):
    ax = axes[i]
    img_tensor = train_dataset[idx]
    
    img_display = img_tensor.permute(1, 2, 0) * 0.5 + 0.5
    ax.imshow(img_display)
    ax.axis('off')

plt.tight_layout()
plt.show()

### Задание 2: Equalized Learning Rate

В стандартных нейросетях мы обычно используем инициализацию весов один раз перед началом обучения, чтобы привести дисперсию активаций к разумным пределам. Однако в `GAN`-ах, веса могут со временем сильно изменяться, что приводит к нестабильности градиентов.

Авторы ProGAN предложили использовать **Equalized Learning Rate** — механизм, который динамически контролирует масштаб весов в процессе работы сети.

Основным фактором нестабильности является $\text{fan\_in}$, или количество входных связей для каждого нейрона. Чем больше $\text{fan\_in}$, тем сильнее «взрывается» сигнал. Для компенсации этого эффекта было предложено использовать коэффициент масштаба $c$:

$$c = \sqrt{2 / \text{fan\_in}}$$

На практике механизм работает так:

1. Инициализируем веса из нормального распределения $\mathcal{N}(0, 1)$.

2. Не меняем веса при инициализации, а вычисляем масштабирующий коэффициент $c$ на основе $\text{fan\_in}$.

3. Во время каждого прохода мы умножаем наши веса на этот коэффициент

Реализуйте механизм `Equalized Learning Rate` в линейном и свёрточном слоях

In [ ]:
class EqualizedLinear(nn.Module):
    """
    Linear layer with Equalized Learning Rate
    Used in the Mapping Network and for Style Modulation
    Args:
        input_size (int): Size of input sample
        output_size (int): Size of output sample
        gain (float): Activation scaling factor, usually sqrt(2)
        lrmul (float): Learning rate multiplier
    """
    def __init__(self, input_size, output_size, gain=2**0.5, lrmul=1):
        super().__init__()
        
        # Calculate He initialization constant: gain * sqrt(1 / fan_in)
        # For Linear, fan_in = input_size
        he_std = ... 
        
        self.w_mul = he_std * lrmul
        self.b_mul = lrmul
        init_std = 1.0 / lrmul
            
        # Initialize weights with Standard Normal Distribution N(0, 1) scaled by init_std
        self.weight = nn.Parameter(...) 
        self.bias = nn.Parameter(torch.zeros(output_size))

    def forward(self, x):
        # Scale the weights and bias using self.w_mul and self.b_mul before passing to F.linear 
        scaled_weight = ...
        scaled_bias = ...
        return F.linear(x, scaled_weight, scaled_bias)
    

class EqualizedConv2d(nn.Module):
    """
    Convolutional layer with Equalized Learning Rate (EqLR)
    Args:
        input_channels (int): Number of input channels
        output_channels (int): Number of output channels
        kernel_size (int): Kernel size 
        stride (int): Stride of the convolution
        gain (float): Activation scaling factor
        lrmul (float): Learning rate multiplier
        padding (int): Zero-padding
    """
    def __init__(self, input_channels, output_channels, kernel_size, stride=1, gain=2**0.5, lrmul=1.0, padding=None):
        super().__init__()
        
        # Calculate He initialization constant
        # For Conv2d, fan_in = input_channels * kernel_width * kernel_height
        fan_in = ...
        he_std = ...
        
        self.kernel_size = kernel_size
        self.w_mul = he_std * lrmul
        self.b_mul = lrmul
        init_std = 1.0 / lrmul

        # Initialize self.weight with N(0, 1) scaled by init_std
        self.weight = nn.Parameter(...)
        self.bias = nn.Parameter(torch.zeros(output_channels))
            
        self.stride = stride
        self.padding = padding if padding is not None else kernel_size // 2

    def forward(self, x):
        # Scale weights by self.w_mul and bias by self.b_mul
        scaled_weight = ...
        scaled_bias = ...
        return F.conv2d(
            x,
            scaled_weight, 
            scaled_bias,
            stride=self.stride,
            padding=self.padding
        )

### Задание 3: Pixel Normalization Layer, NoiseLayer

#### Pixel Normalization Layer

Изначально архитектуры `GAN` столкнулись с тем, что `Batch Normalization` не работала должным образом из-за того, что размеры батчей часто были слишком малы, а статистика по ним — слишком шумной.

Авторы `ProGAN` предложили использовать **нормализацию по пикселям** (**Pixel Normalization**).

Вместо того чтобы нормализовать данные по всему батчу, мы нормализуем $L_2$-норму вектора признаков вдоль канала для каждого отдельного пикселя. То есть, для каждого $(x, y)$ пикселя мы берем его вектор признаков по $C$ каналам и приводим его длину к единице:

$$b_{x,y} = \frac{a_{x,y}}{\sqrt{\frac{1}{C} \sum_{j=0}^{C-1} (a_{x,y}^j)^2 + \epsilon}}$$

Этот метод стабилизирует модель и особенно важен в самом начале генератора, в `Mapping Network`, где он применяется к входному вектору $\mathbf{z}$.

Реализуйте метод `forward` для слоя `PixelNormLayer`. 

In [ ]:
class PixelNormLayer(nn.Module):
    """
    Pixel Normalization
    Used in the Mapping Network to normalize the magnitude of feature vectors
    """
    def __init__(self, epsilon=1e-8):
        super().__init__()
        self.epsilon = epsilon

    def forward(self, x):
        
        # Calculate Mean Square across the channel dimension (dim=1)
        ms = ...

        # Calculate the inverse Root Mean Square (rRMS), use torch.rsqrt() for computational efficiency

        rrms = ...
        
        # Apply rRMS to the input tensor x
        x_norm = ...
        
        return x_norm

#### NoiseLayer

В предыдущих архитектурах `GAN` вектор $\mathbf{z}$ отвечал сразу и за глобальные характеристики, и за мелкие детали. Поэтому, когда нужно было добавить локальные изменения, сеть пыталась получить их из глобальных характеристик. Это приводило к тому, что мелкие детали выглядели искусственными и не подходили к геометрии лица.

В `StyleGAN` решили эту проблему, введя специальный **слой шума** (**Noise Layer**) после каждого сверточного слоя в сети синтеза.

Это позволило генератору разделить обязанности:

- **Вектор стиля** $\mathbf{w}$: контролирует высокоуровневые атрибуты (поза, возраст)

- **Слой шума**: контролирует мелкие детали, которые не влияют на общую картину

Шум масштабируется с помощью обучаемых весов `self.weight`. Это позволяет сети решать, сколько случайности нужно добавить для каждого конкретного признака.

Ваша задача — реализовать метод `forward` в слое `NoiseLayer`.

In [ ]:
class NoiseLayer(nn.Module):
    """
    Injects per-pixel Gaussian noise, scaled by a learned per-channel weight.
    """
    def __init__(self, channels):
        super().__init__()
        self.weight = nn.Parameter(torch.zeros(channels))
        self.noise = None 

    def forward(self, x, noise=None):
        # x shape: [B, C, H, W]
        
        if noise is None and self.noise is None:
            # Create the noise tensor using torch.randn (noise must be 1-channel for broadcasting)
            noise = ...

        elif noise is None:
            noise = self.noise
        
        # Scale the noise by the learned weight and add to features
       
        return ...

### Задание 4: AdaIN 

Вместо того чтобы просто подавать вектор стиля $\mathbf{w}$ в начале, `StyleGAN` использует его для контроля статистики карты признаков на каждом уровне разрешения. Этот процесс регулировки распределения признаков достигается за счет **адаптивной нормализации** (**Adaptive Instance Normalization**, **AdaIN**).

AdaIN состоит из двух этапов, которые мы реализуем с помощью двух классов:

- **Нормализация** $\mathbf{x}_i \to \mathbf{x}'$: Сначала стирается старый стиль карты признаков, причем каждая карта признаков $\mathbf{x}_i$ нормализуется отдельно. Этот этап реализуется с помощью `Instance Normalization` внутри `StyleBlock`.

- **Трансформация** ($\mathbf{w} \to \mathbf{y} \to \mathbf{b}$): Затем мы внедряем новый стиль. Вектор $\mathbf{w}$ проходит через обучаемый линейный слой и превращается в вектор стиля $\mathbf{y} = (y_s, y_b)$. Эти компоненты используются для масштабирования ($y_{s,i}$) и сдвига ($y_{b,i}$) нормализованного вектора $\mathbf{x}'$. Этот этап реализуется в классе `StyleModulation`.

Формула, определяющая эту операцию:

$$\text{AdaIN}(\mathbf{x}_i, \mathbf{y}) = y_{s,i} \frac{\mathbf{x}_i - \mu(\mathbf{x}_i)}{\sigma(\mathbf{x}_i)} + y_{b,i}$$

#### StyleModulation

Этот класс отвечает за финальную трансформацию. Он берет вектор $\mathbf{w}$, преобразует его в параметры стиля $y_{s,i}$ и $y_{b,i}$ и применяет их к нормализованной карте признаков.

Вам необходимо заполнить пропуски в классе `StyleModulation`.

In [ ]:
class StyleModulation(nn.Module):
    """
    Applies the scale (ys) and bias (yb) parameters derived from w.
    This performs the Adaptive part of the AdaIN operation.
    """
    def __init__(self, latent_size, channels): 
        super().__init__()
        self.lin = EqualizedLinear(latent_size, channels * 2, gain=1.0, lrmul=1.0)

    def forward(self, x, latent):
        style = self.lin(latent)
        
        # Reshape style into [B, 2 (scale/bias), C, 1, 1] for broadcasting
        shape = [-1, 2, x.size(1)] + (x.dim() - 2) * [1]
        style = style.view(shape)
        
        # get ys and yb (ys corresponds to style[:, 0], yb to style[:, 1])
        ys = ...
        yb = ...
        
        #Apply the final AdaIN formula
        return ...

#### StyleBlock

Этот класс управляет последовательностью операций в генераторе. Его основная функция — обеспечить правильную последовательность: 

`Noise` $\to$ `Activation` $\to$ `InstanceNorm` $\to$ `StyleAdjustment`. 

`Instance Norm` стирает старый стиль, подготавливая карту признаков к новому стилю.

In [ ]:
class StyleBlock(nn.Module):
    """
    The full synthesis pipeline: Noise -> Activation -> Norm -> StyleAdjustment.
    Args:
        channels (int): Number of feature channels in the input map
        dlatent_size (int): Dimensionality of the intermediate latent space
        activation_layer (nn.Module): The activation function to use
    """
    def __init__(self, channels, dlatent_size, activation_layer):
        super().__init__()

        layers = []
        
        layers.append(('noise', NoiseLayer(channels)))
        layers.append(('activation', activation_layer))
        layers.append(('instance_norm', nn.InstanceNorm2d(channels)))

        self.main_ops = nn.Sequential(OrderedDict(layers))
        self.style_mod = StyleModulation(latent_size=dlatent_size, channels=channels) 

    def forward(self, x, dlatents_in_slice=None):
        """
        Processes the feature map and applies style adjustment.
        :param x: The input feature map
        :param dlatents_in_slice: A slice of the W vector containing the style parameters for this layer 
        """
        # Get x_norm using main_ops
        x_norm = ...
        
        # Apply Style Modulation
        if self.style_mod is not None:
            return ...
        
        return x

### Задание 5: Minibatch Standard Deviation

Основной проблемой в архитектурах `GAN` является **коллапс мод** (**Mode Collapse**) — генератор находит ограниченный набор изображений, которые успешно обманывают дискриминатор, а затем создаёт новые объекты только из этой моды.

Обычный дискриминатор обрабатывает каждое изображение независимо от других в батче, поэтому он не может оценить, насколько разнообразен весь батч фейковых изображений. 

Идея **Minibatch Standard Deviation** (**MBSD**) была впервые представлена в 2016 году [Salimans et al.](https://arxiv.org/pdf/1606.03498) и была адаптирована для модели `ProGAN`. **MBSD** решает проблему коллапса мод, предоставляя дискриминатору возможность оценивать разнообразие текущего мини-батча.

**MBSD** работает как детектор однообразия, который говорит дискриминатору, насколько сильно отличаются изображения в батче:

- Слой делит мини-батч на небольшие группы изображений

- Для каждого признака $\mathbf{x}$ внутри группы рассчитывается стандартное отклонение $\boldsymbol{\sigma}$ по оси батча. Это показывает, насколько сильно отличается один и тот же признак на разных изображениях

- Все полученные значения $\boldsymbol{\sigma}$ усредняются по пространственным координатам и каналам

- Этот вектор добавляется как новый канал к выходной карте признаков дискриминатора

Вам нужно реализовать шаги для расчета стандартного отклонения $\boldsymbol{\sigma}$, описанные выше.

In [ ]:
class MinibatchStddev(nn.Module):
    """
    Calculates the standard deviation of features across the minibatch group 
    and appends it as a new feature map to discourage mode collapse.
    Args:
        group_size (int): The number of samples to group together for statistics, must be a divisor of the batch size 
        num_new_features (int): Number of new feature maps to append
    """
    def __init__(self, group_size=4, num_new_features=1):
        super().__init__()
        self.group_size = group_size
        self.num_new_features = num_new_features

    def forward(self, x):
        b, c, h, w = x.shape
        group_size = min(self.group_size, b)
        
        # Reshape to [G, M, F, C', H, W] where G is group_size, M is Minibatches, F is num_new_features, C' is Internal channels
        y = x.view(group_size, -1, self.num_new_features, c // self.num_new_features, h, w)
        
        # Calculate the standard deviation sqrt(Mean((y - mean(y))^2) + epsilon)
        y_std = ...

        # Average Std Dev over all non-group dimensions (C', H, W)
        y_avg = ...
        
        # Replicate y_avg over the group dimension and expand to [B, F, H, W]
        y_final = ...
        
        #  Concatenate x and y along the channel dimension
        z = ...
        return z

### Задание 6: Truncation Trick 

Обученная модель лучше всего справляется с генерацией изображений, соответствующих высокой плотности в латентном пространстве, то есть, усредненных или "типичных" лиц. Области с низкой плотностью часто содержат нетипичные, сильно искаженные изображения и артефакты.

**Трюк усечения** (**Truncation Trick**) — это метод, который применяется во время генерации. Его цель — принудительно переместить любой случайно сгенерированный вектор стиля $\mathbf{w}$ из областей с низкой плотностью обратно к среднему вектору стиля $\bar{\mathbf{w}}$. 

С одной стороны, этот механизм повышает качество и уменьшает количество артефактов, а с другой — снижает разнообразие объектов.

Трюк работает следующим образом:

- В процессе обучения мы рассчитываем скользящее среднее $\bar{\mathbf{w}}$ всех сгенерированных векторов стиля $\mathbf{w}$.

- При генерации новый вектор $\mathbf{w}$ смешивается с $\bar{\mathbf{w}}$ с использованием коэффициента $\psi$ (`threshold`):

$$\mathbf{w}' = \bar{\mathbf{w}} + \psi \cdot (\mathbf{w} - \bar{\mathbf{w}})$$

В `StyleGAN` этот трюк применяется только к первым `max_layer` векторам, которые контролируют грубые, глобальные атрибуты.

Ваша задача — реализовать логику **Truncation Trick**.

In [ ]:
class Truncation(nn.Module):
    """
    Implements the Truncation Trick in W-space to bias random samples 
    towards the mean, improving fidelity at the cost of diversity.
    """
    def __init__(self, avg_latent, max_layer=8, threshold=0.7, beta=0.995):
        super().__init__()
        self.max_layer = max_layer
        self.threshold = threshold
        self.beta = beta
        self.register_buffer('avg_latent', avg_latent)

    def update(self, last_avg):
        """
        Calculates the new exponential moving average (EMA) of the W vector.
        :param last_avg: The mean W vector of the current training batch.
        """
        # Implement the EMA : new_avg = beta * old_avg + (1 - beta) * current_avg
        self.avg_latent.copy_(...) 

    def forward(self, x):
        # Apply Truncation formula, use torch.lerp
        interp = ...
        
        # Create the masking tensor (only for layers < max_layer)
        do_trunc = ...
        
        # Apply the mask

        return ...

### Задание 7: Constant Input Block

В отличие от предшествующих моделей, где скрытый вектор $\mathbf{z}$ напрямую подавался в первый слой генератора, `StyleGAN` начинает синтез изображения не с шума, а с обучаемой константы. За это отвечает класс `ConstantInputBlock`.

Обучаемая константа `self.const` выполняет роль холста — обучаемого тензора размера $4 \times 4 \times 512$, который даёт начальную пространственную структуру изображения. Поскольку этот холст уже содержит оптимальную базовую информацию, генератору не нужно тратить вычислительные ресурсы на её формирование из вектора $\mathbf{z}$.

Задача `ConstantInputBlock` сводится к тому, чтобы взять этот обучаемый тензор и начать его стилизацию:

- Сначала исходному тензору применяется первый вектор стиля $\mathbf{w}_0$, что задает грубые, высокоуровневые атрибуты изображения

- Затем следует свертка, а после нее — второе добавление вектора стиля $\mathbf{w}_1$

Вам нужно реализовать логику `ConstantInputBlock`, заполнив пропуски в коде.

In [ ]:
class ConstantInputBlock(nn.Module):
    """
    The initial 4x4 block of the StyleGAN Generator. 
    It replaces the direct Z input with a trainable constant
    """
    def __init__(self, 
                 num_channels, 
                 dlatent_size, 
                 activation_layer):
        
        super().__init__()
        
        # Initialize self.const with a tensor of ones
        self.const = nn.Parameter(...) 
        
        # We also need a bias parameter for the constant input
        self.bias = nn.Parameter(...)
        
        self.style_block1 = StyleBlock(
            channels=num_channels, 
            dlatent_size=dlatent_size, 
            activation_layer=activation_layer
        )
        
        self.conv = EqualizedConv2d(num_channels, num_channels, kernel_size=3, padding=1)
        
        self.style_block2 = StyleBlock(
            channels=num_channels, 
            dlatent_size=dlatent_size,  
            activation_layer=activation_layer
        )

    def forward(self, w_slice):
        # w_slice contains the first two W vectors: [B, 2, D]
        batch_size = w_slice.size(0)

        # Expand the constant to match the batch size [1, C, 4, 4] -> [B, C, 4, 4]
        x = ...
        
        # Add the learned bias
        x = ...

        # Apply the first style w_0
        x = ...
        
        # Pass through convolution
        x = ...
        
        # Apply the second style w1
        x = ...
        
        return x

### Задание 8: Synthesis Block

`SynthesisBlock` — это основной блок, который принимает карту признаков низкого разрешения и выдает карту признаков вдвое большего размера.

В отличие от `ConstantInputBlock`, который только создает заготовку $4 \times 4$, этот блок является повторяющимся циклом синтеза, состоящим из 3 этапов:

1. **Upsampling**

2. **First Convolution and Style Adjustment**

3. **Second Convolution and Final Style Adjustment**

Заметьте, что блок потребляет два вектора стиля на одно разрешение. Это сделано для того, чтобы увеличить емкость и управляемость сети. Первый вектор стиля контролирует признаки сразу после повышения разрешения, а второй — уже обработанные признаки. 

Благодаря такому **Style Mixing** мы можем смешивать атрибуты, взятые с разных разрешений.

#### Upsample, DownSample

Архитектура `StyleGAN` построена на идее **Progressive Growing**, при которой **генератор** и **дискриминатор** работают на разных уровнях разрешения. А для перехода между этими уровнями необходимо измененять размеры карт признаков:

- **Upsampling**: используется генератором для увеличения размера карты признаков

- **Downsampling**: используется дискриминатором для уменьшения размера карты признаков

Реализуйте функционал увеличения и уменьшения разрешения карт признаков, используя функции из `PyTorch`.

In [ ]:
class SynthesisBlock(nn.Module):
    """
    The main repeating block of the Generator pipeline. 
    Doubles the resolution and consumes two W vectors.
    """
    def __init__(self, in_channels, out_channels, dlatent_size, gain, activation_layer):
        super().__init__()

        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')

        self.conv1 = EqualizedConv2d(in_channels, 
                                     out_channels, 
                                     kernel_size=3, 
                                     padding=1, 
                                     gain=gain
                                     )
        
        self.style_block1 = StyleBlock(channels=out_channels, 
                                       dlatent_size=dlatent_size,  
                                       activation_layer=activation_layer
                                       )
        
        self.conv2 = EqualizedConv2d(out_channels, 
                                     out_channels, 
                                     kernel_size=3, 
                                     padding=1, 
                                     gain=gain
                                     )
        self.style_block2 = StyleBlock(channels=out_channels, 
                                       dlatent_size=dlatent_size, 
                                       activation_layer=activation_layer
                                       )

    def forward(self, x, w_slice):
        
        # Apply upsample to the input x
        x = ... 
        # Apply conv1 to the result
        x = ...
        # Apply self.style_block1, using the first w vector slice
        x = ...
        # Apply self.conv2
        x = ...
        # Apply self.style_block2, using the second w vector slice
        x = ...
        
        return x

### Задание 9: Discriminator Output Block

`DiscriminatorOutputBlock` является финальным этапом работы дискриминатора,  выполняя три  действия:

1. Обрабатывает карту признаков через слой `MinibatchStddev` и получает информацию о разнообразии всего батча

2. Выполняет свертку $3 \times 3$, которая объединяет все признаки 

3. Результат свертки проходит через два финальных полносвязных слоя, которые и выдают финальный скор для изображения

Реализуйте метод `forward` в классе `DiscriminatorOutputBlock`.

In [ ]:
class DiscriminatorOutputBlock(nn.Module):
    """
    Final block of the Discriminator. It applies MBSD, the last feature 
    extraction, and converts the 4x4 feature map into a single realism score.

    Args:
            mbstd_group_size (int): Number of samples per group for Minibatch Standard Deviation 
            mbstd_num_features (int): Number of new feature maps to append from MBSD
            in_channels (int): Number of input channels to this block
            intermediate_channels (int): Number of channels in the first fully connected layer
            gain (float): He initialization gain for most layers
            activation_layer (nn.Module): The activation function
            resolution (int): The spatial resolution of the input feature map
            in_channels2 (int, optional): Output channels of the last convolution, defaults to in_channels
            output_features (int): Final output size 
            last_gain (float): Gain for the final output linear layer
    """
    def __init__(self, mbstd_group_size, 
                 mbstd_num_features, 
                 in_channels, 
                 intermediate_channels, 
                 gain, 
                 activation_layer, 
                 resolution=4, 
                 in_channels2=None, 
                 output_features=1, 
                 last_gain=1):
        
        super().__init__()
        
        if in_channels2 is None:
            in_channels2 = in_channels
            
        if mbstd_group_size > 1:
            self.mbstd = MinibatchStddev(mbstd_group_size, mbstd_num_features)
            final_in_channels = in_channels + mbstd_num_features
        else:
            self.mbstd = None
            final_in_channels = in_channels

        self.conv = EqualizedConv2d(final_in_channels, in_channels2, kernel_size=3, padding=1, gain=gain)
        self.act1 = activation_layer
        self.fc1 = EqualizedLinear(in_channels2 * resolution * resolution, intermediate_channels, gain=gain)
        self.act2 = activation_layer
        self.fc2 = EqualizedLinear(intermediate_channels, output_features, gain=last_gain)


    def forward(self, x):    

        if self.mbstd is not None:
            # Apply self.mbstd to the input x
            x = ...
            
        # Apply activation to the result of the convolution
        x = self.act1(self.conv(x))
        
        # Flatten the tensor using torch.flatten()
        x_flat = ...
        
        # Apply the first linear layer, then the second activation
        x = ...
        
        # Return the result of the last linear layer
        return ...